# QT TR Parsing 

In [1]:
%run ../../../forth.py

reDef unknown
reDef -->
p e f o r t h    v1.26
source code http://github.com/hcchengithub/peforth
Type 'peforth.ok()' to enter forth interpreter, 'exit' to come back.



In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
raw_data = pd.read_excel('Copy of Result_Rosa_NB -20210609 Tool.xlsx', sheet_name=0)  # 沒有指定就是 worksheet #0 
raw_data

C:\Users\8304018\AppData\Local\Continuum\anaconda3\envs\peforthDevelop\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  """Entry point for launching an IPython kernel.


,PSR Number,Title,Status,P1(What_Happen),P2(What_Device),tool_P3(What_Environment),P4(Function)
0,BITS454665,mapping Rosa BC project SYS_CHG_MIN_INPUT_CU...,Complete,NaN,NaN,NaN,NaN
1,BITS454673,Wistron - Update HellCat TGL- Intel RST PreOS ...,Complete,NaN,NaN,NaN,NaN
2,BITS454678,[CSMB CY20] Update SET_SYS_DPM_DERATING condition,Complete,NaN,NaN,NaN,NaN
3,BITS454679,CBA-2638: PCR6610/PCR6641 : Enable a BIOSConne...,Complete,NaN,NaN,bios,bios
4,BITS454680,"Cypress CCG5 PD FW Release Update, 01.04.64.30",Complete,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1412,NaN,HCT_3/5U_ENG: SUT show black screen after unpl...,Analyze,"halt, shows in",vga,NaN,NaN
1413,NaN,"HCT_5U_ENG_MDAVT: DG1_SUT show ""No base driver...",WAIT,shows in,dgfx,tool,NaN
1414,NaN,"HCT_5U_ENG_MDAVT: DG1_SUT show ""No base driver...",WAIT,shows in,NaN,tool,NaN
1415,NaN,"HCT_5U_ENG_MDAVT: DG1_SUT show ""No base driver...",WAIT,shows in,dgfx,tool,NaN


In [4]:
# James 有整理好各種 key names 放在另一張 work sheet #1
raw_targets = pd.read_excel('Copy of Result_Rosa_NB -20210609 Tool.xlsx',sheet_name=1)
raw_targets

C:\Users\8304018\AppData\Local\Continuum\anaconda3\envs\peforthDevelop\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  


,P1,P1_Count,P2,P2_Count,P3,P3_Count,P4,P4_Count
0,shows in,368,NaN,774.0,NaN,623.0,NaN,454.0
1,NaN,198,storage,103.0,bios,221.0,s3/s4/s5,200.0
2,halt,124,usb device,78.0,os,193.0,long run,156.0
3,error message,110,type-c,75.0,application,144.0,perfacemace,98.0
4,not match the specification,93,audio,57.0,tool,106.0,bios menu,63.0
...,...,...,...,...,...,...,...,...
184,show 0x806c0,1,NaN,NaN,NaN,NaN,NaN,NaN
185,is same as 50%,1,NaN,NaN,NaN,NaN,NaN,NaN
186,portrait display,1,NaN,NaN,NaN,NaN,NaN,NaN
187,couldn't preview,1,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
devices = raw_targets[['P2','P2_Count']].set_index('P2').to_dict()['P2_Count']
del(devices[np.nan]) # 其中有 NaN 項如上列，要注意。
%f devices --> 1 ( count ) cr
for k,v in devices.items():
    if type(k) == str : 
        %f dup . space 1+ k . space v . cr


devices --> {'storage': 103.0, 'usb device': 78.0, 'type-c': 75.0, 'audio': 57.0, 'mouse/keyboard': 55.0, 'vga': 52.0, 'tbt': 39.0, 'lcd': 35.0, 'battery': 26.0, 'cpu': 25.0, 'dimm': 24.0, 'hdmi': 19.0, 'nic/network': 19.0, 'camera': 18.0, 'lid': 16.0, 'type-c device': 16.0, 'dgfx': 16.0, 'dock': 15.0, 'bluetooth': 14.0, 'tpm': 14.0, 'dp': 13.0, 'touch lcd': 12.0, 'ac adapter': 9.0, 'pen': 8.0, 'odd': 7.0, 'monitor': 6.0, 'wlan': 6.0, 'tbt device': 4.0, 'peg': 3.0, 'mediacard/sd card': 3.0, 'spi': 2.0, 'ec': 2.0, 'fingerprint': 2.0, 'sensors': 2.0, 'sata': 2.0, 'bluetooth device': 1.0, 'uhd lcd': 1.0} (<class 'dict'>)

1 storage 103.0
2 usb device 78.0
3 type-c 75.0
4 audio 57.0
5 mouse/keyboard 55.0
6 vga 52.0
7 tbt 39.0
8 lcd 35.0
9 battery 26.0
10 cpu 25.0
11 dimm 24.0
12 hdmi 19.0
13 nic/network 19.0
14 camera 18.0
15 lid 16.0
16 type-c device 16.0
17 dgfx 16.0
18 dock 15.0
19 bluetooth 14.0
20 tpm 14.0
21 dp 13.0
22 touch lcd 12.0
23 ac adapter 9.0
24 pen 8.0
25 odd 7.0
26 monitor

In [6]:
%f devices count nip -->
%f devices -->
%f devices :> keys() -->

devices count nip --> 37 (<class 'int'>)
devices --> {'storage': 103.0, 'usb device': 78.0, 'type-c': 75.0, 'audio': 57.0, 'mouse/keyboard': 55.0, 'vga': 52.0, 'tbt': 39.0, 'lcd': 35.0, 'battery': 26.0, 'cpu': 25.0, 'dimm': 24.0, 'hdmi': 19.0, 'nic/network': 19.0, 'camera': 18.0, 'lid': 16.0, 'type-c device': 16.0, 'dgfx': 16.0, 'dock': 15.0, 'bluetooth': 14.0, 'tpm': 14.0, 'dp': 13.0, 'touch lcd': 12.0, 'ac adapter': 9.0, 'pen': 8.0, 'odd': 7.0, 'monitor': 6.0, 'wlan': 6.0, 'tbt device': 4.0, 'peg': 3.0, 'mediacard/sd card': 3.0, 'spi': 2.0, 'ec': 2.0, 'fingerprint': 2.0, 'sensors': 2.0, 'sata': 2.0, 'bluetooth device': 1.0, 'uhd lcd': 1.0} (<class 'dict'>)
devices :> keys() --> dict_keys(['storage', 'usb device', 'type-c', 'audio', 'mouse/keyboard', 'vga', 'tbt', 'lcd', 'battery', 'cpu', 'dimm', 'hdmi', 'nic/network', 'camera', 'lid', 'type-c device', 'dgfx', 'dock', 'bluetooth', 'tpm', 'dp', 'touch lcd', 'ac adapter', 'pen', 'odd', 'monitor', 'wlan', 'tbt device', 'peg', 'mediacard/

In [7]:
# 拆分 P2 成個別 device 的 list 準備作為 target column 
classes = []
for dd in raw_data['P2(What_Device)']:
    row = []
    for i in range(len(devices)):
        if type(dd) == str and list(devices.keys())[i] in dd:  # dd 有的是 np.nan 屬 float 要排除
            row.append(1)
        else:
            row.append(0)
    classes.append(row)
%f classes count nip -->
%f raw_data count nip -->
%f classes :> [7] -->

classes count nip --> 1417 (<class 'int'>)
raw_data count nip --> 1417 (<class 'int'>)
classes :> [7] --> [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] (<class 'list'>)


In [8]:
targets = pd.DataFrame(np.array(classes), columns = devices)  # 這裡挺妙，columns 直接餵 devices 即可，不必弄成 list(devices.keys()).
targets

,storage,usb device,type-c,audio,mouse/keyboard,vga,tbt,lcd,battery,cpu,...,tbt device,peg,mediacard/sd card,spi,ec,fingerprint,sensors,sata,bluetooth device,uhd lcd
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1413,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1415,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
data = pd.merge(raw_data['Title'],targets, left_index=True, right_index=True)
data 

,Title,storage,usb device,type-c,audio,mouse/keyboard,vga,tbt,lcd,battery,...,tbt device,peg,mediacard/sd card,spi,ec,fingerprint,sensors,sata,bluetooth device,uhd lcd
0,mapping Rosa BC project SYS_CHG_MIN_INPUT_CU...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Wistron - Update HellCat TGL- Intel RST PreOS ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,[CSMB CY20] Update SET_SYS_DPM_DERATING condition,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,CBA-2638: PCR6610/PCR6641 : Enable a BIOSConne...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Cypress CCG5 PD FW Release Update, 01.04.64.30",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,HCT_3/5U_ENG: SUT show black screen after unpl...,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1413,"HCT_5U_ENG_MDAVT: DG1_SUT show ""No base driver...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1414,"HCT_5U_ENG_MDAVT: DG1_SUT show ""No base driver...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1415,"HCT_5U_ENG_MDAVT: DG1_SUT show ""No base driver...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 對照檢查無誤
data.to_excel("1.xls")

In [20]:
import datarobot as dr

In [21]:
dr.Client(token=peforth.execute('api_token').pop(), endpoint=peforth.execute('endpoint').pop())

In [17]:
minimum_sample = 20

In [ ]:
# project factory 
for k,v in devices.items():
    # if k == 'dimm': continue
    if v >= minimum_sample: 
        project = dr.Project.create(project_name='QT TR Title Parsing-{}'.format(k), sourcedata=data)
        %f project -->
        feature_list = project.create_featurelist(k, ['Title',k])
        project.set_target(target=k, worker_count = '-1', mode=dr.AUTOPILOT_MODE.QUICK, featurelist_id=feature_list.id)
        project.wait_for_autopilot()

project --> Project(QT TR Title Parsing-storage) (<class 'datarobot.models.project.Project'>)
In progress: 8, queued: 0 (waited: 0s)
In progress: 8, queued: 0 (waited: 2s)
In progress: 8, queued: 0 (waited: 4s)
In progress: 8, queued: 0 (waited: 6s)
In progress: 8, queued: 0 (waited: 8s)
In progress: 8, queued: 0 (waited: 12s)
In progress: 8, queued: 0 (waited: 17s)
In progress: 3, queued: 0 (waited: 25s)
In progress: 1, queued: 0 (waited: 40s)
In progress: 0, queued: 0 (waited: 61s)
In progress: 16, queued: 0 (waited: 83s)
In progress: 4, queued: 0 (waited: 106s)
In progress: 2, queued: 0 (waited: 127s)
In progress: 0, queued: 0 (waited: 149s)
In progress: 0, queued: 0 (waited: 170s)
In progress: 0, queued: 0 (waited: 192s)
In progress: 0, queued: 0 (waited: 213s)
In progress: 1, queued: 0 (waited: 234s)
In progress: 1, queued: 0 (waited: 256s)
In progress: 1, queued: 0 (waited: 277s)
In progress: 1, queued: 0 (waited: 299s)
In progress: 1, queued: 0 (waited: 321s)
In progress: 0, que

In [40]:
k = 'dimm'
project = dr.Project.create(project_name='QT TR Title Parsing-{}'.format(k), sourcedata=data)
%f project -->

project --> Project(QT TR Title Parsing-dimm) (<class 'datarobot.models.project.Project'>)


In [42]:
feature_list = project.create_featurelist(k, ['Title',k])

In [44]:
feature_list.id 

'60d1d702702ca3172e9f5e11'

In [45]:
project.set_target(target=k, worker_count = '-1', mode=dr.AUTOPILOT_MODE.QUICK, featurelist_id=feature_list.id)
project.wait_for_autopilot()

In progress: 8, queued: 0 (waited: 0s)
In progress: 8, queued: 0 (waited: 2s)
In progress: 8, queued: 0 (waited: 4s)
In progress: 8, queued: 0 (waited: 6s)
In progress: 8, queued: 0 (waited: 10s)
In progress: 8, queued: 0 (waited: 13s)
In progress: 8, queued: 0 (waited: 18s)
In progress: 1, queued: 0 (waited: 26s)
In progress: 1, queued: 0 (waited: 40s)
In progress: 0, queued: 16 (waited: 62s)
In progress: 0, queued: 0 (waited: 84s)


In [22]:
# 先單項做成功
project = dr.Project.create(project_name='QT TR Title Parsing-dimm', sourcedata=data)

In [18]:
for k,v in devices.items():
    if v >= minimum_sample: 
        %f project_name s" -" + k + . space v . cr

QT TR Title Parsing-storage 103.0
QT TR Title Parsing-usb device 78.0
QT TR Title Parsing-type-c 75.0
QT TR Title Parsing-audio 57.0
QT TR Title Parsing-mouse/keyboard 55.0
QT TR Title Parsing-vga 52.0
QT TR Title Parsing-tbt 39.0
QT TR Title Parsing-lcd 35.0
QT TR Title Parsing-battery 26.0
QT TR Title Parsing-cpu 25.0
QT TR Title Parsing-dimm 24.0


In [19]:
for k,v in devices.items():
    if v >= minimum_sample: 
        print("project = dr.Project.create(project_name='QT TR Title Parsing-{}', sourcedata=data)".format(k))

project = dr.Project.create(project_name='QT TR Title Parsing-storage', sourcedata=data)
project = dr.Project.create(project_name='QT TR Title Parsing-usb device', sourcedata=data)
project = dr.Project.create(project_name='QT TR Title Parsing-type-c', sourcedata=data)
project = dr.Project.create(project_name='QT TR Title Parsing-audio', sourcedata=data)
project = dr.Project.create(project_name='QT TR Title Parsing-mouse/keyboard', sourcedata=data)
project = dr.Project.create(project_name='QT TR Title Parsing-vga', sourcedata=data)
project = dr.Project.create(project_name='QT TR Title Parsing-tbt', sourcedata=data)
project = dr.Project.create(project_name='QT TR Title Parsing-lcd', sourcedata=data)
project = dr.Project.create(project_name='QT TR Title Parsing-battery', sourcedata=data)
project = dr.Project.create(project_name='QT TR Title Parsing-cpu', sourcedata=data)
project = dr.Project.create(project_name='QT TR Title Parsing-dimm', sourcedata=data)


In [37]:
project.wait_for_autopilot()

In progress: 0, queued: 0 (waited: 0s)


In [23]:
project

Project(QT TR Title Parsing-dimm)

In [34]:
project.get_featurelists()

[Featurelist(Raw Features),
 Featurelist(Informative Features),
 Featurelist(dimm)]

In [31]:
project.get_features()

[Feature(Title),
 Feature(ac adapter),
 Feature(audio),
 Feature(battery),
 Feature(bluetooth),
 Feature(bluetooth device),
 Feature(camera),
 Feature(cpu),
 Feature(dgfx),
 Feature(dimm),
 Feature(dock),
 Feature(dp),
 Feature(ec),
 Feature(fingerprint),
 Feature(hdmi),
 Feature(lcd),
 Feature(lid),
 Feature(mediacard/sd card),
 Feature(monitor),
 Feature(mouse/keyboard),
 Feature(nic/network),
 Feature(odd),
 Feature(peg),
 Feature(pen),
 Feature(sata),
 Feature(sensors),
 Feature(spi),
 Feature(storage),
 Feature(tbt),
 Feature(tbt device),
 Feature(touch lcd),
 Feature(tpm),
 Feature(type_c),
 Feature(type_c device),
 Feature(uhd lcd),
 Feature(usb device),
 Feature(vga),
 Feature(wlan)]

In [30]:
project.get_datasets()

[]

In [33]:
project.create_featurelist('dimm', ['Title','dimm'])

Featurelist(dimm)

In [ ]:
>>> dataset.get_featurelists()
[DatasetFeaturelist(Raw Features),
 DatasetFeaturelist(universe),
 DatasetFeaturelist(Informative Features)]

>>> dataset_features = [feature.name for feature in dataset.get_all_features()]
>>> custom_featurelist = dataseta
>>> custom_featurelist
DatasetFeaturelist(Custom Features)

>>> dataset.get_featurelists()
[DatasetFeaturelist(Raw Features),
 DatasetFeaturelist(universe),
 DatasetFeaturelist(Informative Features),
 DatasetFeaturelist(Custom Features)]

In [41]:
%f project :> set_target.__doc__ . cr


        Set target variable of an existing project and begin the autopilot process (unless manual
        mode is specified).

        Target setting is asynchronous process, which means that after
        initial request we will keep polling status of async process
        that is responsible for target setting until it's finished.
        For SDK users this only means that this method might raise
        exceptions related to it's async nature.

        When execution returns to the caller, the autopilot process will already have commenced
        (again, unless manual mode is specified).

        Parameters
        ----------
        target : str, optional
            The name of the target column in the uploaded file. Should not be provided if
            ``unsupervised_mode`` is ``True``.
        mode : str, optional
            You can use ``AUTOPILOT_MODE`` enum to choose between

            * ``AUTOPILOT_MODE.FULL_AUTO``
            * ``AUTOPILOT_MODE.MANUAL``
            * `

In [35]:
project.set_target(  
    target='dimm',
    worker_count = '-1', 
    mode=dr.AUTOPILOT_MODE.QUICK
)


Project(QT TR Title Parsing-dimm)

In [ ]:
%f \ project :> get_featurelists.__doc__ . cr 
%f project :> get_featurelists() ==>

In [ ]:
%f project :> get_featurelists()[4] dir ==>
%f project :> get_featurelists()[4].name -->
%f project :> get_featurelists()[4].id -->
%f project :> get_featurelists()[4].num_models -->
    

In [ ]:
project.get_featurelists()[4].delete(delete_dependencies=True)
    

In [ ]:
%f project :> get_featurelists()[4].delete.__doc__ . cr
    

In [ ]:
%f project :> get_featurelists()[4] dir ==>
%f project :> get_featurelists()[4].name -->
%f project :> get_featurelists()[4].id -->
%f project :> get_featurelists()[4].num_models -->
    